<a href="https://colab.research.google.com/github/LangqingZou/Math-509/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [16]:
columns = ['GFOFAIL','WEALTH','RESILIENCE','BELONG','HISCED','ST004D01T','IC001Q04TA','GCSELFEFF','ESCS','EMOSUPS','ST216Q03HA']
data = pd.read_csv('drive/MyDrive/Math_509_Project/CY07_MSU_STU_QQQ.csv',usecols=columns) 
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17,229,323,871) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ST004D01T,ST216Q03HA,IC001Q04TA,HISCED,ESCS,WEALTH,EMOSUPS,GFOFAIL,RESILIENCE,GCSELFEFF,BELONG
0,2,1,1,6,.6747,-0.0996,0.2127,-0.6870,0.3693,-0.4609,-0.2133
1,2,2,2,2,-.7566,-0.7221,-2.0851,-0.6133,-0.4684,0.7623,-0.0845
2,1,5,3,2,-2.5112,-3.6051,0.6364,-0.6133,-1.5248,-0.9477,-0.0824
3,2,2,1,2,-3.1843,-7.0376,1.0346,-0.6870,-0.0614,0.3313,-0.0358
4,2,2,1,4,-1.7557,-1.8375,-0.6576,0.1097,2.3693,0.6965,0.1704


In [35]:
data.dtypes

ST004D01T      object
ST216Q03HA     object
IC001Q04TA     object
HISCED          int64
ESCS           object
WEALTH        float64
EMOSUPS       float64
GFOFAIL       float64
RESILIENCE    float64
GCSELFEFF     float64
BELONG        float64
dtype: object

In [48]:
df = pd.DataFrame(data)
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna() # drop all nan

df_new = df.copy()
df_new['HISCED'] = df_new['HISCED'].astype(object)

# change all object type to numeric type
df['ESCS'] = pd.to_numeric(df['ESCS'])
df['ST004D01T'] = pd.to_numeric(df['ST004D01T'])
df['ST216Q03HA'] = pd.to_numeric(df['ST216Q03HA'])
df['IC001Q04TA'] = pd.to_numeric(df['IC001Q04TA'])

In [50]:
df_new.dtypes

ST004D01T      object
ST216Q03HA     object
IC001Q04TA     object
HISCED         object
ESCS           object
WEALTH        float64
EMOSUPS       float64
GFOFAIL       float64
RESILIENCE    float64
GCSELFEFF     float64
BELONG        float64
dtype: object

In [51]:
# dealing with the invalid values of 'HISCED'
# in 'HISCED' any value larger than 3 is invalid
df = df.loc[df['IC001Q04TA']<4]

In [56]:
# delete any rows contains value larger than 90
# since any value larger than 90 is invalid
mask = (df < 90)
df = df[mask].dropna()

In [57]:
# save the dataset we need to a csv file
df.to_csv("data_subset.csv")

In [55]:
## we save the dataset with 12 variables into a subset
# data1 = pd.read_csv("data_subset.csv")
# data1

,Unnamed: 0,ST004D01T,ST216Q03HA,IC001Q04TA,HISCED,ESCS,WEALTH,EMOSUPS,GFOFAIL,RESILIENCE,GCSELFEFF,BELONG
0,0,2,1,1,6,0.6747,-0.0996,0.2127,-0.6870,0.3693,-0.4609,-0.2133
1,1,2,2,2,2,-0.7566,-0.7221,-2.0851,-0.6133,-0.4684,0.7623,-0.0845
2,2,1,5,3,2,-2.5112,-3.6051,0.6364,-0.6133,-1.5248,-0.9477,-0.0824
3,3,2,2,1,2,-3.1843,-7.0376,1.0346,-0.6870,-0.0614,0.3313,-0.0358
4,4,2,2,1,4,-1.7557,-1.8375,-0.6576,0.1097,2.3693,0.6965,0.1704
...,...,...,...,...,...,...,...,...,...,...,...,...
327751,606583,1,97,1,6,0.2900,0.7740,-0.6576,0.4637,-0.0614,99.0000,-0.3184
327752,606584,1,97,1,5,-0.3287,-0.1200,-2.0819,1.8905,-1.5350,99.0000,-1.8611
327753,606585,2,97,1,4,-2.0076,-1.3099,1.0346,0.4637,-0.5536,99.0000,-1.4766
327754,606586,1,97,1,4,-0.0691,-0.3349,1.0346,1.1422,-0.1433,99.0000,-1.0491


In [54]:
# check if deleting all nans
df.isna().sum()

ST004D01T     0
ST216Q03HA    0
IC001Q04TA    0
HISCED        0
ESCS          0
WEALTH        0
EMOSUPS       0
GFOFAIL       0
RESILIENCE    0
GCSELFEFF     0
BELONG        0
dtype: int64

In [19]:
# check the correlation among all variables
df.corr()

,CNTRYID,ST004D01T,ST216Q03HA,IC001Q04TA,HISCED,ESCS,WEALTH,EMOSUPS,GFOFAIL,RESILIENCE,GCSELFEFF,BELONG
CNTRYID,1.000000,0.002320,0.067518,0.041323,-0.003797,0.022279,0.002772,-0.175899,-0.064012,-0.027531,0.073130,-0.017096
ST004D01T,0.002320,1.000000,0.030845,0.016945,0.017991,0.007019,0.003932,0.026990,0.038817,0.045057,0.024051,0.042945
ST216Q03HA,0.067518,0.030845,1.000000,0.082816,0.041478,0.053251,0.047262,0.189360,0.370708,0.417498,0.892843,0.292741
IC001Q04TA,0.041323,0.016945,0.082816,1.000000,-0.014564,-0.115117,0.036734,0.156225,0.110368,0.173146,0.088250,0.157020
HISCED,-0.003797,0.017991,0.041478,-0.014564,1.000000,0.125544,0.020998,0.023688,0.028872,0.020600,0.045498,0.023178
ESCS,0.022279,0.007019,0.053251,-0.115117,0.125544,1.000000,0.133504,-0.014591,-0.013095,-0.081590,0.072761,-0.054429
WEALTH,0.002772,0.003932,0.047262,0.036734,0.020998,0.133504,1.000000,0.094620,0.146284,0.059480,0.102727,0.131947
EMOSUPS,-0.175899,0.026990,0.189360,0.156225,0.023688,-0.014591,0.094620,1.000000,0.392395,0.347629,0.158299,0.445227
GFOFAIL,-0.064012,0.038817,0.370708,0.110368,0.028872,-0.013095,0.146284,0.392395,1.000000,0.690327,0.396996,0.611406
RESILIENCE,-0.027531,0.045057,0.417498,0.173146,0.020600,-0.081590,0.059480,0.347629,0.690327,1.000000,0.445109,0.587478


In [20]:
# check for multilinearity among all variables 
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
X = df[columns]
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

       feature       VIF
0      CNTRYID  3.215885
1      GFOFAIL  2.485666
2       WEALTH  1.065464
3   RESILIENCE  2.475585
4       BELONG  2.084944
5       HISCED  1.451131
6    ST004D01T  4.103342
7   IC001Q04TA  2.431903
8    GCSELFEFF  6.753512
9         ESCS  1.128125
10     EMOSUPS  1.617689
11  ST216Q03HA  6.885726


In [21]:
dfDummies = pd.get_dummies(df_new[['ST004D01T', 'ST216Q03HA', 'IC001Q04TA', 'HISCED']], drop_first=True)
dfDummies

,ST004D01T_2,ST004D01T_1,ST004D01T_2,ST216Q03HA_2,ST216Q03HA_3,ST216Q03HA_4,ST216Q03HA_5,ST216Q03HA_97,ST216Q03HA_98,ST216Q03HA_99,...,IC001Q04TA_3,IC001Q04TA_7,IC001Q04TA_9,HISCED_1,HISCED_2,HISCED_3,HISCED_4,HISCED_5,HISCED_6,HISCED_99
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611999,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
612000,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
612001,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
612002,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
dfDummies = pd.get_dummies(df, drop_first=True)
# df = pd.concat([df, dfDummies], axis=1)
# df.columns
dfDummies.columns
# plt.figure(figsize=(14,12))
# sns.heatmap(dfDummies.corr(),linewidths=.1,cmap="YlGnBu", annot=True)
# plt.yticks(rotation=0);
# print(dfDummies)


Index(['CNTRYID', 'ST004D01T', 'ST216Q03HA', 'IC001Q04TA', 'HISCED', 'ESCS',
       'WEALTH', 'EMOSUPS', 'GFOFAIL', 'RESILIENCE', 'GCSELFEFF', 'BELONG'],
      dtype='object')

In [ ]:
# TO DO
# standarlized the dataset
scaler = StandardScaler()
scaler.fit(df)
df = scaler.transform(df)
df

In [ ]:
s = pd.Series(list('abca'))
s

0    a
1    b
2    c
3    a
dtype: object